# Load Data

In [185]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
import itertools

In [186]:
idx = pd.read_csv('res_10.csv')


In [187]:
idx['ID'].unique()

array(['LUACTRUU Index', 'SPX Index', 'BCOM Index', 'LBUSTRUU Index',
       'LF98TRUU Index', 'LUATTRUU Index', 'LUMSTRUU Index',
       'LBEATREU Index', 'LP01TREU Index', 'LG30TRUU Index',
       'EMUSTRUU Index', 'LGTRTRUU Index', 'LGDRTRUU Index',
       'LUGCTRUU Index', 'LP06TREU Index', 'LF94TRUU Index',
       'LACHTRUU Index', 'LD08TRUU Index', 'LC07TRUU Index',
       'USYC2Y10 Index', 'LECPTREU Index'], dtype=object)

In [188]:
def return_correlation_ts(df,idx_1, idx_2, corr_window=30, ma_window=30):
    combined = pd.DataFrame()
    combined['DATE'] = df[df['ID'] == idx_1]['DATE']
    combined[idx_1] = df[df['ID'] == idx_1]['PX_LAST'].values
    combined[idx_2] = df[df['ID'] == idx_2]['PX_LAST'].values
    combined = combined[combined[idx_1].notna()]
    combined = combined[combined[idx_2].notna()]
    combined[f'corr_{idx_1}_{idx_2}'] = combined[idx_1].rolling(corr_window).corr(combined[idx_2])
    combined[f'MA_{ma_window}'] = combined[f'corr_{idx_1}_{idx_2}'].rolling(ma_window).mean()
    combined.index = pd.to_datetime(combined['DATE'].values)
    return combined

corr = return_correlation_ts(idx,'SPX Index', 'LUACTRUU Index')

In [189]:
def split_ts(ts, training_fold=2/3):
    train = int(ts.dropna().shape[0]*training_fold)
    y_train = ts.iloc[0:train].dropna()
    y_test = ts.iloc[train+1:-1].dropna()
    y_train = y_train.asfreq('D', method='bfill')
    y_test = y_test.asfreq('D', method='bfill')

    return y_train, y_test


In [190]:
y_train, y_test = split_ts(corr['MA_30'])


# Fit models

In [191]:
from dc_reduced_to_regression import RandomForestGSCV
from sktime.forecasting.base import ForecastingHorizon
from sklearn.ensemble import RandomForestRegressor
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.model_selection import (
    ForecastingGridSearchCV,
    SlidingWindowSplitter,
)



In [192]:
initial_window=int(len(y_train)*0.8)
fh = ForecastingHorizon(y_test.index, is_relative=False)
regressor = RandomForestRegressor(n_jobs=12)
forecaster = make_reduction(regressor, window_length=15, strategy="recursive")
# cv = SlidingWindowSplitter(initial_window=initial_window, window_length=20)
# param_grid = {"window_length": [7, 12, 15]}
# forecaster = ForecastingGridSearchCV(forecaster, strategy="refit", cv=cv, param_grid=param_grid)
forecaster.fit(y=y_train, X=None, fh=fh)
y_pred = forecaster.predict(fh=fh)


/home/viktor/MPhil/m4benchmarkstudy/sktime/utils/validation/series.py:30: FutureWarning: Index.is_integer is deprecated. Use pandas.api.types.is_integer_dtype instead.
  return isinstance(x, pd.Index) and x.is_integer()
/home/viktor/MPhil/m4benchmarkstudy/sktime/utils/validation/series.py:30: FutureWarning: Index.is_integer is deprecated. Use pandas.api.types.is_integer_dtype instead.
  return isinstance(x, pd.Index) and x.is_integer()
/home/viktor/MPhil/m4benchmarkstudy/sktime/utils/validation/series.py:30: FutureWarning: Index.is_integer is deprecated. Use pandas.api.types.is_integer_dtype instead.
  return isinstance(x, pd.Index) and x.is_integer()
/home/viktor/MPhil/m4benchmarkstudy/sktime/utils/validation/series.py:30: FutureWarning: Index.is_integer is deprecated. Use pandas.api.types.is_integer_dtype instead.
  return isinstance(x, pd.Index) and x.is_integer()


TypeError: Index(...) must be called with a collection of some kind, ForecastingHorizon(['2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06',
               '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-10',
               '2020-04-11', '2020-04-12',
               ...
               '2023-09-16', '2023-09-17', '2023-09-18', '2023-09-19',
               '2023-09-20', '2023-09-21', '2023-09-22', '2023-09-23',
               '2023-09-24', '2023-09-25'],
              dtype='datetime64[ns]', length=1271, freq='D', is_relative=False) was passed

In [ ]:
len(y_test)

1271